In [ ]:
# PROYECTO RA1 - FASE 1: Exploración y Limpieza con Pandas

import pandas as pd
import numpy as np

# Cargamos los datos del csv con el read
data_path = "data/gym_lifters.csv"  
df = pd.read_csv(data_path)


print("Dataset cargado correctamente")
print(f"Filas: {df.shape[0]} | Columnas: {df.shape[1]}")
display(df.head())



print("Información general del DataFrame:\n")
df.head()
df.info()

print("Tipos de datos por columna:")
print(df.dtypes)

print("Descripción estadística de columnas numéricas:")
display(df.describe())


print("Valores nulos por columna:")
print(df.isna().sum())

print("Filas duplicadas:", df.duplicated().sum())

# Se eliminan los duplicados si existen
df = df.drop_duplicates()

# Estandarizar nombres de columnas a snake_case
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Eliminar espacios en texto y uniformizar mayúsculas/minúsculas
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].str.strip().str.lower()

# Limpiar valores problemáticos antes de convertir a numérico
# Reemplazar valores inválidos comunes por NaN
valores_invalidos = ['?', 'n/a', 'na', 'unknown', '-1', 'none']
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].replace(valores_invalidos, np.nan)

# Limpiar unidades de texto en columnas numéricas (ej: "154kg" -> 154)
columnas_peso = ['snatch_kg', 'clean_and_jerk_kg', 'total_kg', 'body_weight_kg']
for col in columnas_peso:
    if col in df.columns:
        # Extraer solo números, manejar casos como "154kg", "300kg", etc.
        df[col] = df[col].astype(str).str.extract(r'(\d+\.?\d*)', expand=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Convertir columnas numéricas a tipos apropiados
# Age
df["age"] = pd.to_numeric(df["age"], errors='coerce')

# Year (año)
df["year"] = pd.to_numeric(df["year"], errors='coerce')
# Limpiar años inválidos (ej: 9999, años futuros)
df["year"] = df["year"].where((df["year"] >= 1900) & (df["year"] <= 2025), np.nan)

# Event rank
df["event_rank"] = pd.to_numeric(df["event_rank"], errors='coerce')

# Tratar valores faltantes en columnas numéricas
# Age: rellenar con mediana
df["age"] = df["age"].fillna(df["age"].median())

# Pesos: rellenar con mediana de cada columna
for col in columnas_peso:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

# Year: rellenar con moda (año más común)
df["year"] = df["year"].fillna(df["year"].mode()[0] if not df["year"].mode().empty else np.nan)

# Convertir columnas a tipos de datos adecuados
print("\n[OK] Dataset limpio:")
print(df.info())

print("\n[INFO] Tipos de datos finales:")
print(df.dtypes)

print("\n[INFO] Valores nulos después de limpieza:")
print(df.isna().sum())

print("\n[INFO] Resumen estadístico de columnas numéricas:")
display(df.describe())

# Vista final de los datos
display(df.sample(5))

# Guardar el DataFrame limpio en un nuevo archivo CSV
clean_path = "data/gym_lifters_clean.csv"
df.to_csv(clean_path, index=False)
print(f"Dataset limpio guardado en: {clean_path}")


FileNotFoundError: [Errno 2] No such file or directory: 'gym_lifters.csv'